In [ ]:
!pip -q install langchain tiktoken keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 95.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
import keras_nlp
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en"
)

497986112/497986112 [==============================] - 13s 0us/step


In [ ]:
import keras_nlp
gpt2_llm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_extra_large_en"
)

6231301960/6231301960 [==============================] - 75s 0us/step


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate



In [ ]:
text_splitter = CharacterTextSplitter()

In [ ]:
# load the doc
with open('/content/state_of_the_union.txt') as f:
    how_to_win_friends = f.read()
texts = text_splitter.split_text(how_to_win_friends)

In [ ]:
len(texts)

11

In [ ]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:3]]

In [ ]:
docs

In [ ]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.3 MB/s eta 0:00:00


In [ ]:
import torch
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any


# set context window size
context_window = 2048
# set number of output tokens
num_output = 256

# store the pipeline/model outisde of the LLM class to avoid memory issues
model_name = "gpt2"

class CustomLLM(LLM):

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt_length = len(prompt)
        response = gpt2_llm.generate(prompt)

        # only return newly generated tokens
        return response[prompt_length:]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": model_name}

    @property
    def _llm_type(self) -> str:
        return "custom"


In [ ]:
chain = load_summarize_chain(CustomLLM(),
                             chain_type="map_reduce")


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

   1.) The United States and Europe are imposing sanctions on Russian officials and companies, and
the Russian Central Bank is restricting the access of its banks to the international financial
system.   2.) Russia's economy has been hit hard by sanctions. The ruble has dropped 40%, inflation
is running at 15%, and Russia has lost its place as the world's top energy producer.   3.) Putin's
response has been to escalate the military conflict in Ukraine, and he has sent troops into Crimea
and Eastern Ukraine.   4.) Russia's economy is in a deep depression. The ruble has dropped,
inflation has skyrocketed, and Russia's GDP has been cut in half.   5.) The Russian military is
invading Eastern Ukraine.   6.) Putin is a Russian dictator who wants to conquer the Ukraine.   7.)
Putin is a dictator who is occupying Ukraine.  8.) Putin is a tyrant.   9.) Putin is a tyrant.
Putin is a dictator.  He is invading Ukraine.  He is occupying Ukraine.  He is destroying Ukraine.
Putin is destroying Europe